In [ ]:
# Set TensorFlow version to 1.x
%tensorflow_version 1.x

# Print assigned GPU
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
    print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
    print('and then re-execute this cell.')
else:
    print(gpu_info)

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
# IMPORTANT: Ensure that the repository directory follows the path:
# "/content/gdrive/MyDrive/Colab Data/COCO-Human-Pose"

In [ ]:
# Download dataset to pre-specified folder on local VM instance
# Change dir to project dir
%cd "/content/gdrive/MyDrive/Colab Data/COCO-Human-Pose"

import time
from datetime import datetime, timedelta

start = time.time()

!bash "./scripts/coco_dl.sh" /content/datasets

download_time = time.time() - start
print("Total download time: {}".format(str(timedelta(seconds=download_time))))

In [ ]:
!ls /content/datasets/coco/train2017 | wc -l
!ls /content/datasets/coco/val2017 | wc -l

In [ ]:
# Ensure that filesystem is set up correctly and will not be the bottleneck
import time
from datetime import datetime, timedelta

start = time.time()

import os
def print_files_in_dir(path):
    print(len(os.listdir(path)), " files at ", path)

print_files_in_dir('/content/datasets/coco/train2017/')
print_files_in_dir('/content/datasets/coco/val2017/')


setup_time = time.time() - start
print("Total setup time: {}".format(str(timedelta(seconds=setup_time))))

if setup_time > 1:
    print("There appears to be a bottleneck with filesystem loading time. This may severely impact training speed.")

In [ ]:
# Toggle COLAB_TRAINING variable in constants file
# https://askubuntu.com/questions/20414/find-and-replace-text-within-a-file-using-commands
!sed -i.bak 's/COLAB_TRAINING = False/COLAB_TRAINING = True/g' constants.py 

# Remove backup file
!rm constants.py.bak

# Upgrade required imgaug package because of a bug on 0.2.9
!pip install --upgrade imgaug==0.4.0

In [ ]:
!python3 train.py --epochs 100 --batch 16 --hourglass 4 --subset 1.0 --loss "weighted_mse" --augment "light"

# To resume, add the following to the above command, and point it to the subdirectory of the model checkpoints. Ensure everything matches
# --resume True --resume-epoch 42 --resume-subdir "2021-03-14-15h-92m_batchsize_12_hg_4"

# The options are below. NOTE that ALL options must match across training sessions:
# --batch           I recommend 12 for hg 8, 16 for hg 4
# --augment         none, light, medium, heavy
# --loss            keras_mse, weighted_mse, euclidean_loss, focal_loss
# --activation      linear, sigmoid, relu
# --optimizer       rmsProp, adam
# --learning-rate    we originally used 5e-4. One of the pytorch models used 1e-2. Currently default is 5e-3

# NOTE about subset: if you decide to run on a subset <1.0, there is no functionality to retrieve the same subset of data if the model is 
# stopped then resumed. For this reason, subset should only be used for quick tests rather than reporting robust/repeatable results. 
# Resuming a model with subset data may be meaningless because it is not guaranteed to get the same subset of data. It is more equivalent 
# to starting a brand new training session with slightly pre-trained weights.

In [ ]:
# %% Create Eval Instance
import time
from datetime import datetime, timedelta

import imp
import evaluation_wrapper
imp.reload(evaluation_wrapper)
import evaluation
imp.reload(evaluation)
import constants
import matplotlib.pyplot as plt
%matplotlib inline

eval = evaluation_wrapper.EvaluationWrapper('2021-04-03-18h-11m_batchsize_16_hg_4_loss_keras_mse_aug_medium_sigma4_learningrate_5.0e-03_opt_adam_gt-4kp_activ_sigmoid')
# eval = evaluation_wrapper.EvaluationWrapper('2021-04-01-21h-59m_batchsize_16_hg_4_loss_weighted_mse_aug_light_sigma4_learningrate_5.0e-03_opt_rmsProp_gt-4kp_activ_sigmoid_subset_0.50_lrfix')

In [ ]:
# %% Run OKS
start = time.time()

eval.calculateOKS(1, constants.Generator.representative_set_gen, average_flip_prediction=True)
elapsed = time.time() - start
print("Total OKS average normal & flip time: {}".format(str(timedelta(seconds=elapsed))))

start = time.time()

eval.calculateOKS(1, constants.Generator.representative_set_gen, average_flip_prediction=False)

elapsed = time.time() - start
print("Total OKS time: {}".format(str(timedelta(seconds=elapsed))))

In [ ]:
# %% Run PCK
start = time.time()

eval.calculatePCK(1, constants.Generator.representative_set_gen, average_flip_prediction=True)
eval.calculatePCK(1, constants.Generator.representative_set_gen, average_flip_prediction=False)

elapsed = time.time() - start
print("Total PCK time: {}".format(str(timedelta(seconds=elapsed))))

In [ ]:
# %% Visualize representative batch heatmaps
start = time.time()

eval.visualizeHeatmaps(constants.Generator.representative_set_gen)

elapsed = time.time() - start
print("Total heatmap time: {}".format(str(timedelta(seconds=elapsed))))


In [ ]:
# %% Visualize representative batch keypoints
start = time.time()

eval.visualizeKeypoints(constants.Generator.representative_set_gen)

elapsed = time.time() - start
print("Total keypoint time: {}".format(str(timedelta(seconds=elapsed))))